# Can AI preserve our science legacy

In [ ]:
import nltk
import pandas as pd
import gensim.corpora 
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize , sent_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import string 
import gensim
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string , preprocess_documents
from gensim.utils import lemmatize

In [ ]:
import requests
import json

In [ ]:
api_root = 'https://ntrs.nasa.gov'

In [ ]:
def getDocsLinks(count = 100):
  offset = 0
  url = api_root + f'/api/citations/search?center=CDMS&page.size=100&offset={offset}'

  download_urls = []
  while len(download_urls) < count:
     resp = requests.get(url).json()
     #print(resp['results'][-1]['downloads'][0]['links']['fulltext'])
     downloadable_files = [ ( i['id'] ,i['downloads'][0]['links']['fulltext'] )
                          for i in resp['results'] 
                          if len(i['downloads']) > 0]
     download_urls.extend(downloadable_files)
     offset += 100
     return download_urls[:count]


In [ ]:
def getDocsLinkByID(id):

 


   download_url = None
   download_url = requests.get(api_root + f'/api/citations/{id}/downloads/{id}.txt')


   return download_url


In [ ]:
docs = getDocsLinks(count = 5)

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
texts = [ (i[0], requests.get(api_root + i[1]).content, ) for i in docs ]

In [ ]:
import re
import string
import glob

from gensim import utils
from gensim.parsing.porter import PorterStemmer


# improved list from Stone, Denis, Kwantes (2010)
STOPWORDS = """
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves
"""
STOPWORDS = frozenset(w for w in STOPWORDS.split() if w)

def remove_stopwords(s):
    s = utils.to_unicode(s)
    return " ".join(w for w in s.split() if w not in STOPWORDS)
RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)
def strip_punctuation(s):
    s = utils.to_unicode(s)
    return RE_PUNCT.sub(" ", s)

strip_punctuation2 = strip_punctuation

RE_TAGS = re.compile(r"<([^>]+)>", re.UNICODE)
def strip_tags(s):
    s = utils.to_unicode(s)
    return RE_TAGS.sub("",s)



def strip_short(s, minsize=3):
    s = utils.to_unicode(s)
    return " ".join(e for e in s.split() if len(e) >= minsize)



RE_NUMERIC = re.compile(r"[0-9]+", re.UNICODE)
def strip_numeric(s):
    s = utils.to_unicode(s)
    return RE_NUMERIC.sub("", s)



RE_NONALPHA = re.compile(r"\W", re.UNICODE)
def strip_non_alphanum(s):
    s = utils.to_unicode(s)
    return RE_NONALPHA.sub(" ", s)



RE_WHITESPACE = re.compile(r"(\s)+", re.UNICODE)
def strip_multiple_whitespaces(s):
    s = utils.to_unicode(s)
    return RE_WHITESPACE.sub(" ", s)

RE_AL_NUM = re.compile(r"([a-z]+)([0-9]+)", flags=re.UNICODE)
RE_NUM_AL = re.compile(r"([0-9]+)([a-z]+)", flags=re.UNICODE)
def split_alphanum(s):
    s = utils.to_unicode(s)
    s = RE_AL_NUM.sub(r"\1 \2", s)
    return RE_NUM_AL.sub(r"\1 \2", s)



def stem_text(text):
    """
    Return lowercase and (porter-)stemmed version of string `text`.
    """
    text = utils.to_unicode(text)
    p = PorterStemmer()
    return ' '.join(p.stem(word) for word in text.split())

stem = stem_text

DEFAULT_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces,
                   strip_numeric, remove_stopwords, strip_short]


def preprocess_string(s, filters=DEFAULT_FILTERS):
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s.split()


In [ ]:
dataSet = list()
for i in range(0,99):
   text = str(texts[i][1])
   dataSet.append(preprocess_string(text.replace('\\n', ' ').replace('-',' ')))

In [ ]:
def read(docNumber):
 docs = getDocsLinks(docNumber)
 texts = [ (i[0], requests.get(api_root + i[1]).content, ) for i in docs ]

 return texts

In [ ]:
def preprocessing(count):
 docs = getDocsLinks(count)
 #requests.get(api_root + 'api/citations/{id}/downloads/{id}.txt')
 texts = [ (i[0], requests.get(api_root + i[1]).content, ) for i in docs ]
 dataSet = list()
 for text in texts:
   text = str(text[1])
   dataSet.append(preprocess_string(text.replace('\\n', ' ').replace('-',' ')))

 lemmatizer = WordNetLemmatizer()

 for doc in dataSet:
   doc = [lemmatizer.lemmatize(word) for word in doc]  

 

 return dataSet  

In [ ]:
def testingPreprocessing(id):
  doc = getDocsLinkByID(id)
  doc = str(doc.content)
  data = list()
  data.append(preprocess_string(doc.replace('\\n', ' ').replace('-',' ')))

  lemmatizer = WordNetLemmatizer()
  for doc in data:
   doc = [lemmatizer.lemmatize(word) for word in doc]  
  return data

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

for doc in dataSet:
  doc = [lemmatizer.lemmatize(word) for word in doc]

In [ ]:
def doc2Bow(dataSet):
  dictionary = gensim.corpora.Dictionary(dataSet)
  bow_corpus = [dictionary.doc2bow(doc) for doc in dataSet]

  return bow_corpus , dictionary

In [ ]:
import gensim.corpora 
dictionary = gensim.corpora.Dictionary(dataSet)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in dataSet]

In [ ]:
dataSet = preprocessing(100000)
bow_corpus , dictionary = doc2Bow(dataSet)
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 5, 
                                   id2word = dictionary,                                    
                                   )


In [ ]:
lda_model.print_topics()

[(0,
  '0.008*"sec" + 0.006*"data" + 0.005*"figure" + 0.005*"pressure" + 0.004*"temperature" + 0.004*"time" + 0.004*"control" + 0.003*"inlet" + 0.003*"test" + 0.003*"space"'),
 (1,
  '0.005*"figure" + 0.005*"data" + 0.003*"flight" + 0.003*"nasa" + 0.003*"control" + 0.003*"temperature" + 0.003*"level" + 0.003*"time" + 0.003*"pressure" + 0.003*"robot"'),
 (2,
  '0.006*"figure" + 0.004*"data" + 0.004*"sec" + 0.003*"model" + 0.003*"time" + 0.003*"control" + 0.003*"space" + 0.003*"based" + 0.002*"pressure" + 0.002*"nasa"'),
 (3,
  '0.007*"data" + 0.005*"figure" + 0.004*"time" + 0.004*"nasa" + 0.004*"pressure" + 0.003*"cloud" + 0.003*"flow" + 0.003*"temperature" + 0.003*"test" + 0.003*"surface"'),
 (4,
  '0.008*"data" + 0.007*"cloud" + 0.006*"figure" + 0.004*"temperature" + 0.003*"time" + 0.003*"control" + 0.003*"surface" + 0.003*"nasa" + 0.002*"flight" + 0.002*"results"')]

In [ ]:
testData = testingPreprocessing(19900018794)
test_bow_corpus , test_dictionary = doc2Bow(testData)

for index  in sorted(lda_model[test_bow_corpus]):
  print(lda_model.print_topic(index[0][0]))

0.006*"figure" + 0.004*"data" + 0.004*"sec" + 0.003*"model" + 0.003*"time" + 0.003*"control" + 0.003*"space" + 0.003*"based" + 0.002*"pressure" + 0.002*"nasa"


In [ ]:
import pickle

In [ ]:
filename = 'finalized_keyWords_model.sav'
pickle.dump(lda_model, open(filename, 'wb'))

In [ ]:
l = pickle.load(open(filename, 'rb'))
l.print_topics()

[(0,
  '0.008*"sec" + 0.006*"data" + 0.005*"figure" + 0.005*"pressure" + 0.004*"temperature" + 0.004*"time" + 0.004*"control" + 0.003*"inlet" + 0.003*"test" + 0.003*"space"'),
 (1,
  '0.005*"figure" + 0.005*"data" + 0.003*"flight" + 0.003*"nasa" + 0.003*"control" + 0.003*"temperature" + 0.003*"level" + 0.003*"time" + 0.003*"pressure" + 0.003*"robot"'),
 (2,
  '0.006*"figure" + 0.004*"data" + 0.004*"sec" + 0.003*"model" + 0.003*"time" + 0.003*"control" + 0.003*"space" + 0.003*"based" + 0.002*"pressure" + 0.002*"nasa"'),
 (3,
  '0.007*"data" + 0.005*"figure" + 0.004*"time" + 0.004*"nasa" + 0.004*"pressure" + 0.003*"cloud" + 0.003*"flow" + 0.003*"temperature" + 0.003*"test" + 0.003*"surface"'),
 (4,
  '0.008*"data" + 0.007*"cloud" + 0.006*"figure" + 0.004*"temperature" + 0.003*"time" + 0.003*"control" + 0.003*"surface" + 0.003*"nasa" + 0.002*"flight" + 0.002*"results"')]

In [ ]:
results = [WordNetLemmatizer().lemmatize(i,'v')
           for i in words 
           if len(i) > 3 and 
           i not in gensim.parsing.preprocessing.STOPWORDS]

In [ ]:
results

In [ ]:
dic = gensim.corpora.Dictionary([results,])

In [ ]:
bow = dic.doc2bow(results)

In [ ]:
print(dic.items())

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f6bf1334d50>)


In [ ]:
[[(dic[id], count) for id, count in line] for line in bow]

TypeError: ignored

In [ ]:
[ for i in bow]

AttributeError: ignored

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install rake_nltk
from rake_nltk import Rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
texts[1][1].decode('utf-8')

'I\ni\n\n\'\n\n.\n\nN A S A TN D-5515\n- -\n\nNASA TECHNICAL NOTE\n\n4. /\n\nLOAN COPY: RETURN TO\nAFWL ( W W - 2 )\nKllRTLAND AFB, N MEX\n\nTHE EFFECT OF\nOBJECT MOTION\nI N FRAUNHOFERHOLOGRAPHY WITH\nAPPLICATION TO VELOCITY\nMEASUREMENTS\n\nby WiZZium P. Dotson, Jr.\nMan fled Spacecraft Cejzter\nHoustou, Texus\n\nN A T I O N AA E R O N A U T I C S N D P A C E D M I N I S T R A T I O N\nL\nA S\nA\n\nW A S H I N G T O N , D. C.\n\nNOVEMBER 1969\n1\n\nTECH LIBRARY KAFB, NM\n\n1\nI\nI\n\nI. R E P O R T NO.\n\n2.\n\nI\n\nNO.\n\n1\n\nNASA T N D-5515\n4 . T I T L E AND\n\nG O V E R N M E N TA C C E S S I O N\n\nSUBTITLE\n\n3 RECIPIENT\'SCATALOG\n.\n\nI\nT November 1969\n5.\n\nTHE EFFECT OF OBJECT MOTION IN FRAUNHOFER\nHOLOGRAPHY WITH APPLICATION TO VELOCITY\nMEASUREMENTS\n\nREPORTDATE\n\n6.\n\n7 . AUTHOR(5)\n\nIO.\n\nt\n\nManned Spacecraft Center\nHouston, Texas 77058\n\n11.\n\nC O N T R A C T OR G R A N T NO.\n\nR E P O R TT Y P EA N DP E R I O DC O V E R E D\n\nTechnical Note\n\nNational 

In [ ]:
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(str(texts[0][1]))
keyword_extracted = rake_nltk_var.get_ranked_phrases()[:5]
print(keyword_extracted)


['nh .\\ nh .\\ nc .\\ nt .\\ nj .\\ ne .\\ nh .\\ nf .\\ nn .\\ n', 'nm .\\ nw .\\ nj .\\ ne .\\ nj .\\ nf .\\ nj .\\ nh .\\ ne .\\ n', 'nw .\\ nd .\\ nb .\\ nr .\\ nj .\\ ng .\\ ni .\\ nr .\\ n', "bellcomm .\\ ninc .\\ n9 5 5 l \\' enfant", 'niv .\\ nv .\\ nvi .\\ nvii .\\ n']


# Summarization

In [ ]:
resp = requests.get(api_root + '/api/citations/19810002806').json()

In [ ]:
doc = str(requests.get(api_root + resp['downloads'][0]['links']['fulltext']).content)

In [ ]:
import gensim
from gensim.summarization import summarize

In [ ]:
print(summarize(doc,0.01), sep='\n\n')

Type of Report and Period covered\n\nTRW Defense and Space Systems Group\nPower Conversion Electronics Department\nOne Space Park\nRedondo Beach, California\t 90278\t\n_\n12, Sponsoring Ap ncy Nema and Address\n\nFinal TECH.\nJune 1976- Jan. 1980\n14, Sponsoring Apmcy code\n\nNASA Lewis Research Center\n21000 Brookpark Road\nC leveland, Ohio\t 44135\n\n15, Supplementary Notes\n\nNASA Technical Monitor:\t\n\n18,\n\n4\n\nJoseph Kolecki\n\nAbstract\nThree basic switchin g regulators:\t\n\nbuck, boost, and buck/boost, employing a Standardized\n\nControl Module (SCM) were characterized by a common small signal block diagram in Volume I of\n\nthe report, "Application Handbook for a Standardized Control Module for DC-DC Converters,"\nEmploying the unified model, regulator performance such as stability, aud\'iosusceptibility,\n\noutput impedance, and step-load transient were analyzed and key performance indices were\nexpressed in simple analytical forms,\t It has been demonstrated that the per

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.1 MB/s 
     |████████████████████████████████| 163 kB 70.4 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 12.4 MB/s 


In [ ]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

TypeError: ignored

In [ ]:
T5Tokenizer.from_

transformers.utils.dummy_sentencepiece_objects.T5Tokenizer